<a href="https://colab.research.google.com/github/YunusKaratepe/duygu-tanima/blob/main/BP_CNN_Ilker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import keras
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras import activations

# Load Data From Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls
!pwd

drive  sample_data


# Get Data Set

In [ ]:
def getDataset(foldNumber): 
  directory = '/content/drive/My Drive/BPDatasetV3
  image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

  train_dataset = image_generator.flow_from_directory(batch_size=32,
                                                  directory=directory,
                                                  shuffle=True,
                                                  target_size=(128, 128), 
                                                  subset="training",
                                                  class_mode='categorical')

  validation_dataset = image_generator.flow_from_directory(batch_size=32,
                                                  directory=directory,
                                                  shuffle=True,
                                                  target_size=(128, 128), 
                                                  subset="validation",
                                                  class_mode='categorical')


  train_set=tf.keras.preprocessing.image_dataset_from_directory(
      directory,
      labels="inferred",
      label_mode="categorical",
      class_names=None,
      batch_size=32,
      image_size=(128, 128),
      interpolation="lanczos5"
  )
  return train_set



# Training Custom CNN

In [5]:
def trainCustomCNN():
  model = keras.Sequential() 
  model.add(keras.layers.Conv1D(128, input_shape=[X_train.shape[1], 1], kernel_size=80, strides=4, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001)))
  model.add(keras.layers.BatchNormalization()) 
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling1D(pool_size=4, strides=None)) 
  model.add(keras.layers.Conv1D(128, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001)))
  model.add(keras.layers.BatchNormalization()) 
  model.add(keras.layers.Activation('relu')) 
  model.add(keras.layers.MaxPooling1D(pool_size=4, strides=None)) 
  model.add(keras.layers.Conv1D(256, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling1D(pool_size=4, strides=None))
  model.add(keras.layers.Conv1D(512, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001))) 
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling1D(pool_size=4, strides=None))
  model.add(keras.layers.Lambda(lambda x : keras.backend.mean(x, axis=1))) 
  model.add(keras.layers.Dense(10, activation='softmax')) 
  model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy', keras.metrics.TruePositives(), keras.metrics.TrueNegatives(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives()]) 
  model.summary()

NameError: ignored

# Training CNN With Transfer Learning

In [ ]:
def transferLearning():  
  #Set pretrained Layers
  #for layer in model.layers[:150]:
  #    layer.trainable=False
  #for layer in model.layers[150:]:
  #    layer.trainable=True

  #opt = tf.keras.optimizers.SGD(learning_rate=0.001)


  opt  = tf.keras.optimizers.Adam(learning_rate=0.00001)
  for j in range(3,4):
    foldNo = 'fold' + str(j + 1)
    test_set  = testPreprocess(foldNo)
    train_set = trainPreprocess(foldNo)
    print("")
    print(foldNo)
    
    epochNo = 100     
    base_model = tf.keras.applications.ResNet101(
    include_top=False, weights='imagenet', classes=3)
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    preds=Dense(3,activation='softmax')(x) #final layer with softmax activation
    model=Model(inputs=base_model.input,outputs=preds)


    print("Epoch: " + str(epochNo) + "\n\n")
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.TruePositives(), keras.metrics.TrueNegatives(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives()])
    model.fit(x = train_set, validation_data = test_set, epochs = epochNo)
    print("Egitim basariyla tamamlandi.")

# Main

In [ ]:

trainCustomCNN()

# Making Single Prediction

In [ ]:
img_path = '/content/drive/My Drive/Hushem set1/fold1/train/02_Tapered/image_053.BMP'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/drive/My Drive/Set1_5xAugmented/fold3/test/04_Amorphous/image_032.BMP', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.argmax(result))
print(train_set.class_names)